# Tinyflux: Example with fictious bank transactions

In [1]:
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta

## Initialize the TinyFlux database

In [2]:
db = TinyFlux("swiss_bank_transactions.db")

## Writing a CSV file into Tinyflux

In [3]:
# Read CSV and insert data into TinyFlux
with open("example_data/swiss_bank_transactions.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"),
            measurement="transactions",
            fields={"amount": float(row["amount"])},  # Only numeric fields
            tags={
                "location": row["location"],
                "account_id": row["account_id"],
                "currency": row["currency"],  # Move non-numeric fields to tags
                "transaction_type": row["transaction_type"]
            }
        )
        db.insert(point)

## Writing manually into Tinyflux

In [4]:
# Create manual records
p1 = Point(
    time=datetime(2024, 1, 1, 12, 0, tzinfo=timezone.utc),
    tags={"location": "Winterthur", "account_id": "1234", "currency": "CHF", "transaction_type": "deposit"},
    fields={"amount": 1000.0}
)

p2 = Point(
    time=datetime(2024, 1, 2, 12, 0, tzinfo=timezone.utc),
    tags={"location": "Winterthur", "account_id": "1234", "currency": "CHF", "transaction_type": "deposit"},
    fields={"amount": 2000.0}
)

# Insert into the DB.
db.insert_multiple([p1, p2])

2

## Querying by Time

In [12]:
# Search for a time value
Time = TimeQuery()
time_start = Time >= datetime(2023, 1, 1, tzinfo=timezone.utc)
time_end = Time < datetime(2023, 12, 31, tzinfo=timezone.utc)
time_count = db.count(time_start & time_end)
print("\nTime Query Count:")
print(time_count)



Time Query Count:
1000


## Querying by Field

In [8]:
# Search for a field value
Field = FieldQuery()
field_results = db.select("fields.amount", Field.amount > 1000.0)
print("\nField Query Results:")
for result in field_results:
    print(result)


Field Query Results:
2434.85
3856.44
3630.08
3635.79
4938.55
1469.45
3339.61
1457.41
4263.39
2719.0
4416.31
2902.73
3341.11
3532.26
4676.52
2023.05
1313.0
1265.39
3093.07
1246.52
4217.86
1388.59
1854.59
2237.1
3436.11
1375.51
4173.0
2231.75
4621.64
1220.63
1134.37
3298.77
3298.72
3636.69
3388.31
2598.6
1127.82
3095.35
4706.89
1319.81
3186.9
1483.23
4338.68
2274.74
1369.3
2008.56
2332.77
2644.16
3110.68
1828.2
1122.21
1385.27
4143.9
3539.9
1410.38
1149.24
3809.07
1164.47
3705.17
4530.88
4138.06
2075.53
3378.07
2001.15
3682.32
4736.98
1205.31
1917.85
2858.53
2444.53
1795.53
1037.79
4365.69
1164.32
4685.33
4623.68
2041.01
3487.21
1446.53
3743.79
1694.49
4991.53
2910.86
2518.31
3471.45
1781.12
4940.18
3523.26
3810.91
2607.64
4034.04
3060.22
2934.63
3188.49
1868.8
3352.21
2634.2
4659.49
3600.29
3254.54
4596.11
1484.39
3415.86
1757.59
1157.36
1012.41
4612.04
2004.46
1040.23
3903.76
4484.26
4698.15
4821.97
1851.57
2230.68
1173.87
1155.54
3823.65
1674.92
4932.97
3942.9
4653.99
1013.69
1568.23

## Querying by Tag

In [10]:
# Search for a tag value
Tag = TagQuery()
tag_results = db.search(Tag.location == 'Winterthur')
print("Tag Query Results:")
for result in tag_results:
    print(result)

Tag Query Results:
Point(time=2024-01-01T12:00:00+00:00, measurement=_default, tags=location:Winterthur; account_id:1234; currency:CHF; transaction_type:deposit, fields=amount:1000.0)
Point(time=2024-01-02T12:00:00+00:00, measurement=_default, tags=location:Winterthur; account_id:1234; currency:CHF; transaction_type:deposit, fields=amount:2000.0)
